# Delta Method

This program defines a class to implement the delta method and demonstrates usage with examples.
The objective is to provide a tool similar to `nlcom` in Stata (https://www.stata.com/manuals/rnlcom.pdf) that can be easily applied to results obtained from Statsmodels (https://www.statsmodels.org/stable/index.html). Calculations in this notebook are confirmed against results from Stata file `Delta Method Examples.do`.

## Overview 

Suppose $B$ is a consistent estimator of the vector $\beta\in\mathbb{R}^m$. Then, as the sample size used to compute $B$ increases:
$$\sqrt{n}(B - \beta)\rightarrow N(0,\Sigma),$$
where $\Sigma$ is the $m\times m$ covariance matrix of the elements of $B$. Let $g: \mathbb{R}^m \rightarrow \mathbb{R}^l$, for some $l\geqslant 1$. Then, according to the delta method:
$$\sqrt{n}(g(B) - g(\beta))\rightarrow N(0,\nabla g(\beta)^T\cdot \Sigma \cdot \nabla g(\beta)),$$
Given $B$, $\Sigma$, and a function $g$, `delta_method` computes the point estimate and standard error of $g(B)$, the z-score, p-value, and a confidence interval around the point estimate. Read more about the delta method here: https://en.wikipedia.org/wiki/Delta_method.

## Installation

Installation
Install the module from PyPI with the shell command:

``pip install fredpy``

Or download the source here: https://github.com/letsgoexploring/delta-method/blob/gh-pages/dist/delta_method-0.1.tar.gz

## Preliminaries

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

from delta_method import delta_method

## Examples

Data for the examples is produced using the Notebook `Inflation and Unemployment Data.ipynb`. The estimated models are based on examples in Chapter 14 from Hansen's textbook *Econometrics*. The examples will show a couple different options.

The first four examples are based on estimates of this equation:

\begin{align}
\Delta \pi_{t} & = \beta_0 + \beta_1\Delta \pi_{t-1} + \beta_2\Delta \pi_{t-2} + \epsilon_t
\end{align}

The examples then use the delta method to estimate several functions of the parameter estimates $\beta_0$, $\beta_1$, and $\beta_2$.

First, load data containing quarterly observations of inflation and unemployment. 

In [2]:
# Load data
data = pd.read_csv('inflation_unemployment_data.csv',index_col = 1,parse_dates=True)

# Compute first difference of inflation and lagged first differences
data['inflation_diff'] = data['inflation'].diff()
data['L1.inflation_diff'] = data['inflation_diff'].shift(1)
data['L2.inflation_diff'] = data['inflation_diff'].shift(2)

# Drop missing values that result from lagging.
data = data.dropna()

# Form OLS model in Statsmodels and estimate
Y = data['inflation_diff']
X = sm.add_constant(data[['L1.inflation_diff','L2.inflation_diff']])

model = sm.OLS(Y,X)
sm_results = model.fit()

sm_results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         inflation_diff   R-squared:                       0.182
Model:                            OLS   Adj. R-squared:                  0.176
Method:                 Least Squares   F-statistic:                     28.56
Date:                Mon, 29 Jul 2024   Prob (F-statistic):           6.38e-12
Time:                        14:58:44   Log-Likelihood:                -180.78
No. Observations:                 259   AIC:                             367.6
Df Residuals:                     256   BIC:                             378.2
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                 0.0025      0.030      0.081      0.935      -0.057       0.062
L1.inflation_diff    -0.3850      0.059     -6.523      0.000      -0.501      -0.269
L2.inflation_diff    -0.3272      0.059     -5.542      0.000      -0.444      -0.211
==============================================================================
Omnibus:                      139.073   Durbin-Watson:                   1.992
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1737.556
Skew:                          -1.825   Prob(JB):                         0.00
Kurtosis:                      15.152   Cond. No.                         2.21
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Example 1:

A function of a single parameter:
\begin{align}
g(\beta) & = \beta_0^{0.25}
\end{align}

First, define function $g$:

In [3]:
def g(beta,param_names=None):
    
    if param_names is not None:
        beta = pd.Series(beta,index=param_names)
    
    return beta['const']**0.25

In [4]:
# delta method using statsmodels results
dm = delta_method(g=g,sm_results=sm_results)

dm.results

,Coefficient,Std. err.,z,P>|z|,95% ci [upper],95% ci [lower]
g_1,0.223004,0.685164,0.325476,0.744821,-1.119892,1.565901


Alternatively, directly input parameter vector and variance-covariance matrix

In [5]:
# Recover coefficient and variance-covariance estimates from statsmodels results
beta=sm_results.params
param_names = sm_results.params.index
vcov = sm_results.cov_params().to_numpy()

# delta method
dm = delta_method(beta=beta,g=g,vcov=vcov,g_kwargs={'param_names':param_names})

dm.results

,Coefficient,Std. err.,z,P>|z|,95% ci [upper],95% ci [lower]
g_1,0.223004,0.685164,0.325476,0.744821,-1.119892,1.565901


Note that since only one element of $\beta$ is used in $g$, `beta` and `vcov` could be specified as scalars:

In [6]:
# Recover coefficient and variance-covariance estimates from statsmodels results
beta=beta['const']
param_names = ['const']
vcov = sm_results.cov_params().to_numpy()[0][0]

# delta method
dm = delta_method(beta=beta,g=g,vcov=vcov,g_kwargs={'param_names':param_names})

dm.results

,Coefficient,Std. err.,z,P>|z|,95% ci [upper],95% ci [lower]
g_1,0.223004,0.685164,0.325476,0.744821,-1.119892,1.565901


### Example 2:

A nonlinear function of parameters:
\begin{align}
g(\beta) & = \frac{\beta_1}{\beta_2}
\end{align}

Define function $g$:

In [7]:
def g(beta):
    
    return beta[1]/beta[2]

vcov = sm_results.cov_params().to_numpy()

dm = delta_method(beta=sm_results.params,g=g,vcov=vcov)

dm.results

,Coefficient,Std. err.,z,P>|z|,95% ci [upper],95% ci [lower]
g_1,1.176394,0.235405,4.997311,5.813527e-07,0.715008,1.637779


### Example 3:

Multiple nonlinear functions of parameters:
\begin{align}
g(\beta) & = \left(\beta_1 + \beta_2, \frac{\beta_1}{\beta_2}, \beta_0 + \frac{\beta_1}{\beta_2^{6}}\right)
\end{align}

Define function $g$:

In [8]:
def g(beta):
    
    return beta[1] + beta[2], beta[1]/beta[2], beta[0] + beta[1]/beta[2]**6

vcov = sm_results.cov_params().to_numpy()

dm = delta_method(beta=sm_results.params,g=g,vcov=vcov)

dm.results

,Coefficient,Std. err.,z,P>|z|,95% ci [upper],95% ci [lower]
g_1,-0.712195,0.094804,-7.512280,5.810647e-14,-0.898008,-0.526383
g_2,1.176394,0.235405,4.997311,5.813527e-07,0.715008,1.637779
g_3,-313.502465,328.729043,-0.953680,3.402454e-01,-957.799550,330.794621


## Example 4

Different confidence level. Suppose that $g$ is:
\begin{align}
g(\beta) & = \frac{\beta_1}{\beta_2}
\end{align}

Define function $g$:

In [9]:
def g(beta):
    
    return beta[1]/beta[2]

vcov = sm_results.cov_params().to_numpy()

dm = delta_method(beta=sm_results.params,g=g,vcov=vcov,level=0.99)

dm.results

,Coefficient,Std. err.,z,P>|z|,99% ci [upper],99% ci [lower]
g_1,1.176394,0.235405,4.997311,5.813527e-07,0.57003,1.782757


## Example 5

In this final example, we estimate:
\begin{align}
\Delta\pi_{t} & = \beta_0 + \beta_1\Delta\pi_{t-1} + \beta_2\Delta\pi_{t-2} + \beta_3\Delta\pi_{t-3} + \beta_4\Delta\pi_{t-4} + \beta_5u_{t-1} + \beta_6u_{t-2} + \beta_7u_{t-3} + \beta_8u_{t-4} + \epsilon_t
\end{align}
and find the estimate and standard error for the sum of the coefficients on unemployment lags: $\beta_5 + \beta_6 + \beta_7 + \beta_8$

In [10]:
# Load data
data = pd.read_csv('Inflation_Unemployment_Data.csv',index_col = 1,parse_dates=True)

# Compute first difference of inflation and lagged first differences
data['inflation_diff'] = data['inflation'].diff()
data['L1.inflation_diff'] = data['inflation_diff'].shift(1)
data['L2.inflation_diff'] = data['inflation_diff'].shift(2)
data['L3.inflation_diff'] = data['inflation_diff'].shift(3)
data['L4.inflation_diff'] = data['inflation_diff'].shift(4)

# Compute fir
data['L1.unemployment'] = data['unemployment'].shift(1)
data['L2.unemployment'] = data['unemployment'].shift(2)
data['L3.unemployment'] = data['unemployment'].shift(3)
data['L4.unemployment'] = data['unemployment'].shift(4)

data = data.dropna()

# data = data.loc['1961':'2017']

Y = data['inflation_diff']

X = sm.add_constant(data[['L1.inflation_diff','L2.inflation_diff','L3.inflation_diff','L4.inflation_diff','L1.unemployment','L2.unemployment','L3.unemployment','L4.unemployment']])

model = sm.OLS(Y,X)
sm_results = model.fit(cov_type='HAC',cov_kwds={'maxlags':5})
sm_results = model.fit()

sm_results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         inflation_diff   R-squared:                       0.208
Model:                            OLS   Adj. R-squared:                  0.183
Method:                 Least Squares   F-statistic:                     8.147
Date:                Mon, 29 Jul 2024   Prob (F-statistic):           8.95e-10
Time:                        14:58:44   Log-Likelihood:                -175.80
No. Observations:                 257   AIC:                             369.6
Df Residuals:                     248   BIC:                             401.6
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                 0.0056      0.120      0.047      0.963      -0.230       0.241
L1.inflation_diff    -0.3780      0.066     -5.738      0.000      -0.508      -0.248
L2.inflation_diff    -0.3992      0.071     -5.594      0.000      -0.540      -0.259
L3.inflation_diff    -0.0583      0.071     -0.817      0.415      -0.199       0.082
L4.inflation_diff    -0.1587      0.064     -2.482      0.014      -0.285      -0.033
L1.unemployment      -0.0031      0.046     -0.067      0.946      -0.093       0.087
L2.unemployment      -0.0665      0.060     -1.104      0.270      -0.185       0.052
L3.unemployment       0.0716      0.060      1.186      0.237      -0.047       0.190
L4.unemployment      -0.0024      0.045     -0.053      0.958      -0.092       0.087
==============================================================================
Omnibus:                      132.195   Durbin-Watson:                   2.001
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1502.402
Skew:                          -1.753   Prob(JB):                         0.00
Kurtosis:                      14.314   Cond. No.                         49.2
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [11]:
def g(beta,param_names=None):
    
    beta = pd.Series(beta,index=param_names)
    
    return beta['L1.unemployment'] + beta['L2.unemployment'] + beta['L3.unemployment'] + beta['L4.unemployment']

dm = delta_method(g=g,sm_results=sm_results)

dm.results

,Coefficient,Std. err.,z,P>|z|,95% ci [upper],95% ci [lower]
g_1,-0.000409,0.019538,-0.020953,0.983283,-0.038703,0.037885
